In [3]:
!pip install webdriver-manager


  Using cached webdriver_manager-4.0.1-py2.py3-none-any.whl.metadata (12 kB)
Using cached webdriver_manager-4.0.1-py2.py3-none-any.whl (27 kB)


In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

# Setup WebDriver using WebDriver Manager
service = Service(ChromeDriverManager().install())
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)

# Open the HTML file
driver.get("https://www.naturalreaders.com/online/")  # Update this path

# JavaScript to capture and download audio
js_code = """
async function startRecording() {
    const stream = await navigator.mediaDevices.getUserMedia({ audio: true });
    const mediaRecorder = new MediaRecorder(stream);
    let audioChunks = [];

    mediaRecorder.ondataavailable = event => {
        audioChunks.push(event.data);
    };

    mediaRecorder.onstop = async () => {
        const audioBlob = new Blob(audioChunks, { type: 'audio/wav' });
        const audioUrl = URL.createObjectURL(audioBlob);
        const a = document.createElement('a');
        a.style.display = 'none';
        a.href = audioUrl;
        a.download = 'audio.wav';
        document.body.appendChild(a);
        a.click();
        window.URL.revokeObjectURL(audioUrl);
    };

    mediaRecorder.start();
    console.log('Recording started');

    setTimeout(() => {
        mediaRecorder.stop();
        console.log('Recording stopped');
    }, 10000); // Record for 10 seconds
}

startRecording();
"""


In [ ]:

# Inject the JavaScript code into the webpage
driver.execute_script(js_code)

# Wait for the recording to complete and download to finish
time.sleep(15)

# Close the browser
driver.quit()

# Open the HTML file
